# IMPORTS

In [1]:
import json
import pandas as pd
import subprocess as SB
import xml.etree.ElementTree as ET

In [2]:
data = pd.read_csv('nixon_single.csv', sep='\t', encoding = 'utf-8')
data.head()

FileNotFoundError: [Errno 2] File b'nixon_single.csv' does not exist: b'nixon_single.csv'

# HELPERS

In [ ]:
"""
    Returns the result of the method evaluate(...), which is the middle part of the html
    @text_review: The original text review
    @shap_string The shap (json) information as a string
"""
def getMiddle(text_review, shap_string):
    result = evaluate(text_review, shap_string)
    return result

In [ ]:
"""
    Turns a string into JSON
    @shap_string : The shap (json) information as a string
"""
def parseShap(shap_string):
    return json.loads(shap_string)

In [ ]:
"""
    Checks if a word has a shap value
    @list : Shap information
    @term : A term as it appears in the original review text
"""
def find_dictionary(list, term):    
    for x in range(0, len(list)):
        if (list[x]['term'] == term.lower()) or (term[0:-1].lower() == list[x]['term'] and term[-1] == 's'):
            return list[x]
        elif (term[-3:] == "n't" and term[0:-2].lower() == list[x]['term']):
            return list[x]
        elif (term.lower() == "watches") and (list[x]['term'] == 'watch'):
            return list[x]
    
    return None

In [ ]:
"""
    Returns the html code with highlights around 
    @input : the original review text
    @shap : the shap json, containing shap information for the review text
"""
def evaluate(input, shap):
    parts = custom_split(input)
    output = ""
    for part in parts:
        parse = parseShap(shap)
        s = find_dictionary(parse, part)
        if s is not None:
            element = invoke_paragraph(s, part)
            output = output + element
        else: output = output + part
    return output

In [ ]:
"""
    Returns the review text split in parts based on special signs
"""
def custom_split(input):
    array = ['.', '?', '!', ',', ' ', '-', '"']
    result = []
    length = len(input)

    word = ''

    for i in range(len(input)):
        char = input[i]
        if char in array:
            if word != '' :
                result.append(word)
                word = ""
            result.append(char)
        else:
            word += char
            
    if word != ' ' : 
        result.append(word)

    return result

In [ ]:
def invoke_paragraph(dictionary, part):
    factor = dictionary['factor']
    identifier = dictionary['id']
    orientation = dictionary['orientation']
        
    return highlight_part(factor, identifier, orientation, part)

In [ ]:
"""
    Creates a text highlighting and returns the html code for the text highlighting
"""
def highlight_part(shapvalue, identifier, orientation, term) :
    factor = round(abs(shapvalue),2)
    orientation = lambda x : "positive" if (x > 0) else "negative"
    
    return "<span class=\"highlighter " + identifier + "\" data-shap=\"" + str(factor) + "\" data-orientation=\"" + orientation(shapvalue) + "\">" + term + "</span>";

In [ ]:
def getOrientation(shap_json):
    empty = {}
    for el in shap_json:
        term = el['term']
        orientation = el['orientation']
        empty[term] = orientation
    return empty

In [ ]:
def getIDs(shap_json):
    empty = {}
    for el in shap_json:
        term = el['term']
        identifier = el['id']
        empty[term] = identifier
    return empty

In [ ]:
def getFactors(shap_json):
    empty = {}
    for el in shap_json:
        term = el["term"]
        factor = el["factor"]
        empty[term] = factor
    return empty

In [ ]:
def getOccurrence(shap_json):
    empty = {}
    for el in shap_json:
        term = el["term"]
        factor = el["occurrence"]
        empty[term] = factor
    return empty

## GENERATE INPUT FILE cloud.jar

In [ ]:
def createInputFile(factors, occurrences, filename):
    file = filename + ".txt"
    words = ""
    for key in factors:
        occ = occurrences[key]
        if (occ != 0):
            v = factors[key]
            size = (float(v) * 100)    
            for i in range(1, int(size)):
                words += key + " "
            words += ". "

    with open(file, "w") as text_file:
        text_file.write(words)

## GENERATE SVG

In [ ]:
def generateSVG(filename):
    file = filename + ".txt"
    args = ['java', '-jar', 'cloudy.jar', '-w1200', '-h500', '-Lmds', '-ps', '-pg', '-pn', '-O', file]
    SB.call(args)

## PARSE SVG

In [ ]:
def parseSVG(filename, ids, orientations):
    file = filename + ".svg"
    root = ET.parse(file).getroot()
    internal = root[1]
    result = '<svg width="500" height="275" font-family="Arial" fill="#a9a9a9" font-style="normal" font-weight="bold"> <g transform="scale(0.5)">'

    for child in internal:
        attributes = child.attrib
        term = child[0]

        transform = attributes['transform']
        x = term.attrib['x']
        y = term.attrib['y']
        size = attributes['font-size']
        text = term.text
        orientation = orientations[term.text]

        fill = 'black'
        if (orientation == 'positive'):
            fill = 'rgb(44, 160, 44)'
        elif (orientation == 'negative'):
            fill = 'rgb(214, 39, 40)'

        id_tag = ids[term.text]
        new_tag = '<g transform="' + transform + '" font-size="' + str(size) + '" fill="' + fill + '" onclick="highlight(`' + id_tag + '`, `' + orientation + '`)' +'">' + '<text x="' + x + '" y="' + y + '">' + term.text + '</text></g>'
        result = result + new_tag
    result = result + '</g></svg>'
    return result

# RUN SCRIPT

In [ ]:
result_list = []
filename = "test_cloud"
html_col = []

header = '<p id="review_text" class="review_result">'
footer = '</p>'

for i in data.index:
    row = data.loc[i]
    
    # SETUP
    shap = row.Shap_json    
    review_text = row.Review
       
    # HIGHLIGHTS IN REVIEW TEXT
    middle = getMiddle(review_text, shap)
    html = header + middle + footer
    html_col.append(html)
        
    # CREATE INPUT FILE (word cloud)
    s = json.loads(shap)
    factors = getFactors(s)
    occurrences = getOccurrence(s)
    createInputFile(factors, occurrences, filename)
    
    # CREATE WORD CLOUDS
    generateSVG(filename)
    orientations = getOrientation(s)
    ids = getIDs(s)
    result = parseSVG(filename, ids, orientations)
    result_list.append(result)
    
    # RESET
    shap = ""
    review_text = ""

In [ ]:
data["Highlights"] = html_col
data["Cloud"] = result_list
del data['Prediction']
del data['Shap_json']

data.to_csv('nixon_html.csv', sep='\t', index=False)
data

In [ ]:
data.to_json('nixon.json', orient='records')